This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = '' #load from json file below

In [2]:
!ls

api_data_wrangling_mini_project.html   quandl_config_secret.json
api_data_wrangling_mini_project.ipynb


In [4]:
import json
with open('./quandl_config_secret.json') as key_file:
    key = json.load(key_file)
API_KEY = key['key']

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [5]:
# First, import the relevant modules
import requests
import json

### Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse into the JSON structure that will be returned

In [26]:
# url = 'https://www.quandl.com/api/v3/datasets/'
# params = {'database_code': 'FSE', 'dataset_code': 'AFX_X', 'api_key': API_KEY}
url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json'
params = {'api_key': API_KEY, 'start_date': '2017-01-01', 'end_date': '2017-12-31'}

#create a request
r = requests.get(url, params=params)

In [27]:
print(r.url)
print(r.status_code)

https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?api_key=2gxUki2FGMDKbxSrweuE&start_date=2017-01-01&end_date=2017-12-31
200


In [28]:
content = json.loads(r.content)
dataset = content['dataset_data']
print(dataset.keys())
dataset['data'][0]

dict_keys(['limit', 'transform', 'column_index', 'column_names', 'start_date', 'end_date', 'frequency', 'data', 'collapse', 'order'])


['2017-12-29',
 51.76,
 51.94,
 51.45,
 51.76,
 None,
 34640.0,
 1792304.0,
 None,
 None,
 None]

In [29]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure

# Already Done Above


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

## My Implementation

In [31]:
# 1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
# Already Done Above
data = dataset['data']
print(dataset.keys())
# print(data)

# def data_for_2017(data_full):
#     data = []
#     for d in data_full:
#         if '2017' in d[0]:
#             data.append(d)
#     return data
# data = data_for_2017(data)

dict_keys(['limit', 'transform', 'column_index', 'column_names', 'start_date', 'end_date', 'frequency', 'data', 'collapse', 'order'])


In [32]:
# 2. Convert the returned JSON object into a Python dictionary.
# Already done above using json.loads()
print(dataset.keys())
print(dataset['column_names'])
# # print(dataset['column_index'])
# print(dataset['start_date'])
print(dataset['data'][0])
DATE_IDX = 0
OPEN_IDX = 1
HIGH_IDX = 2
LOW_IDX = 3
CLOSE_IDX = 4
VOL_IDX = 6

dict_keys(['limit', 'transform', 'column_index', 'column_names', 'start_date', 'end_date', 'frequency', 'data', 'collapse', 'order'])
['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']
['2017-12-29', 51.76, 51.94, 51.45, 51.76, None, 34640.0, 1792304.0, None, None, None]


In [33]:
# 3. Calculate what the highest and lowest opening prices were for the stock in this period.
def calc_highest_lowest_opening_prices(data):
    max_ = -float('inf')
    min_ = -max_
    for d in data:
        if d[OPEN_IDX] and d[OPEN_IDX] < min_: min_ = d[OPEN_IDX]
        elif d[OPEN_IDX] and d[OPEN_IDX] >  max_: max_ = d[OPEN_IDX]
    print(f'Highest opening price is {max_} and Lowest opening price is {min_}')
calc_highest_lowest_opening_prices(data)

Highest opening price is 53.11 and Lowest opening price is 34.0


In [34]:
# 4. What was the largest change in any one day (based on High and Low price)?
def largest_change_any_one_day(data):
    max_chng = -float('inf')
    idx = None
    for i, d in enumerate(data):
        high = d[HIGH_IDX]
        low = d[LOW_IDX]
        if high and low:
            if max_chng < abs(high-low): max_chng = abs(high-low); idx=i
    print(f'Largest daily price change is {max_chng} and occurs on {data[idx][0]}') 
largest_change_any_one_day(data)      
        

Largest daily price change is 2.8100000000000023 and occurs on 2017-05-11


In [35]:
# 5. What was the largest change between any two days (based on Closing Price)?
def largest_change_between_two_days(data):
    max_diff = abs(data[0][CLOSE_IDX] - data[1][CLOSE_IDX])
    idx = 0
    for i in range(2, len(data)):
        prev = data[i-1]
        curr = data[i]
        if curr[CLOSE_IDX] and prev[CLOSE_IDX]: diff = abs(curr[CLOSE_IDX] - prev[CLOSE_IDX])
        if diff > max_diff: max_diff=diff; idx = i
    print(f'Largest change between any two days is {max_diff} and it occurs between {data[idx][0]} and {data[idx-1][0]}')
largest_change_between_two_days(data)
        

Largest change between any two days is 2.559999999999995 and it occurs between 2017-08-08 and 2017-08-09


In [36]:
# 6. What was the average daily trading volume during this year?
def avg_trading_vol(data):
    count = 0
    sum_vol = 0
    for d in data:
        if d[VOL_IDX]:
            count += 1
            sum_vol += d[VOL_IDX]
    print(f'Average daily trading volume is {sum_vol/count}')

avg_trading_vol(data)    

Average daily trading volume is 89124.33725490196


In [37]:
# 7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)
def median_trading_vol(data):
    vol = []
    for d in data:
        if d[VOL_IDX]: vol.append(d[VOL_IDX])
    vol = sorted(vol)
    if len(vol)%2 == 0:
        p2 = len(vol)/2
        p1 = p2-1
        median = (vol[p1]+vol[p2])/2
    else:
        p = len(vol)//2
        median = vol[p]
    print(f'The median trading volume is: {median}')

median_trading_vol(data)            

The median trading volume is: 76286.0
